In [13]:
import os
import numpy as np
from scipy.io import loadmat

folder_path = r"..\los_data"  # 你的文件夹路径
file_list = [f for f in os.listdir(folder_path) if f.endswith('.mat')]
file_list = sorted(file_list)

# 选择处理逻辑：
# 1 -> 先差分再转置
# 2 -> 先转置再差分
process_mode = 1  # 改成 2 可对比另一个顺序

data = []

for i, file_name in enumerate(file_list):
    full_path = os.path.join(folder_path, file_name)
    mat_contents = loadmat(full_path)

    if 'data_Ineed' in mat_contents:
        raw_data = mat_contents['data_Ineed']

        if process_mode == 1:
            # 先差分再转置
            diff_data = np.empty_like(raw_data)
            diff_data[0] = raw_data[0]
            diff_data[1:] = raw_data[1:] - raw_data[:-1]
            processed_data = diff_data.T

            # 按 320 个信号重构
            # num_signals, num_samples = processed_data.shape
            # block_size = 320
            # num_blocks = num_signals // block_size
            # processed_data = processed_data[:num_blocks * block_size, :]
            # processed_data = processed_data.reshape((block_size, num_blocks * num_samples)).T

        elif process_mode == 2:
            # 先转置
            transposed = raw_data.T

            # 按 320 个信号重构
            num_signals, num_samples = transposed.shape
            block_size = 320
            num_blocks = num_signals // block_size
            reshaped_data = transposed[:num_blocks * block_size, :]
            reshaped_data = reshaped_data.reshape((block_size, num_blocks * num_samples)).T

            # 转置，让时间轴在 axis=0（和 process_mode=1 对齐）
            reshaped_data_T = reshaped_data.T

            # 差分
            diff_data = np.empty_like(reshaped_data_T)
            diff_data[0] = reshaped_data_T[0]
            diff_data[1:] = reshaped_data_T[1:] - reshaped_data_T[:-1]

            # 再转置回来
            processed_data = diff_data.T


        else:
            raise ValueError("process_mode 必须是 1 或 2")

        data.append(processed_data)
    else:
        print(f"Warning: {file_name} 里没有 'data_Ineed' 变量！")

# 打印检查
for i, d in enumerate(data):
    print(f"第{i}个文件（{file_list[i]}）的数据 shape (处理后): {d.shape}, 前5个值: {d[:5,0]}")


第0个文件（dev0.mat）的数据 shape (处理后): (7014, 320), 前5个值: [-0.01318091-0.00598181j -0.00856645-0.0066582j  -0.01293569+0.00314399j
 -0.00608675-0.00281703j -0.00453262-0.00123845j]
第1个文件（dev1.mat）的数据 shape (处理后): (6632, 320), 前5个值: [-0.05595885-0.01681558j -0.04190274-0.01379795j -0.02817297-0.00111078j
 -0.02078103-0.00585922j -0.00773754-0.00191155j]
第2个文件（dev10.mat）的数据 shape (处理后): (8935, 320), 前5个值: [-0.07714929-0.02866302j -0.07859567-0.01964287j -0.07749676-0.0211156j
 -0.04533909-0.0204815j  -0.05166423-0.02073226j]
第3个文件（dev11.mat）的数据 shape (处理后): (7776, 320), 前5个值: [ 1.71212724e-03+0.00604403j -9.08998346e-05+0.01620005j
 -2.51134976e-02-0.00123921j -5.13536853e-03-0.01767933j
  7.84350184e-03+0.01182339j]
第4个文件（dev13.mat）的数据 shape (处理后): (7410, 320), 前5个值: [-0.02351107-0.00558919j -0.0092649 -0.01073413j -0.01412528-0.00914526j
 -0.01682761-0.00316691j -0.01362142-0.00254273j]
第5个文件（dev14.mat）的数据 shape (处理后): (6274, 320), 前5个值: [ 0.41253131+0.17627416j  0.2342232 +0.34427072j -0.361

In [14]:
import numpy as np

# 假设data是从之前的加载代码中获取的
# data = <从前一部分代码中加载的数据>

SNR_dB = 20  # 设定信噪比（dB）

noisy_data = []  # 用来存储加噪声后的数据

for d in data:
    # 计算信号的标准差
    signal_std = np.std(d)
    
    # 计算噪声的标准差，根据信噪比（SNR）
    noise_std = signal_std / (10 ** (SNR_dB / 20))  # 根据 SNR(dB) 计算噪声标准差
    
    # 生成高斯噪声并加到原信号上
    noise = np.random.normal(0, noise_std, d.shape)  # 均值为0，标准差为噪声标准差
    noisy_signal = d + noise  # 添加噪声
    
    noisy_data.append(noisy_signal)  # 保存加噪声后的信号

# 打印检查加噪声后的数据
print("原始数据和加噪声数据示例：")
print(f"原始信号样本（第一个信号）：{data[0][0][:10]}")  # 打印前10个采样点
print(f"加噪声信号样本（第一个信号）：{noisy_data[0][0][:10]}")  # 打印前10个采样点


原始数据和加噪声数据示例：
原始信号样本（第一个信号）：[-1.31809103e-02-5.98180779e-03j -6.15371749e-04+1.05275085e-03j
 -4.19571718e-03+6.31124165e-03j -1.67451997e-01+2.43141602e-01j
 -4.31873701e-01+7.30005434e-01j  4.64915904e-01+1.16523036e-02j
  1.02535875e+00-1.29181797e+00j -6.75687685e-01-3.50605527e-01j
 -1.28489375e-01+1.07416726e+00j  8.85266615e-01-2.39137528e-01j]
加噪声信号样本（第一个信号）：[ 0.03945283-5.98180779e-03j  0.0208486 +1.05275085e-03j
 -0.14489092+6.31124165e-03j -0.1003895 +2.43141602e-01j
 -0.51642852+7.30005434e-01j  0.43478773+1.16523036e-02j
  1.12421639-1.29181797e+00j -0.75446404-3.50605527e-01j
 -0.25473892+1.07416726e+00j  0.81246553-2.39137528e-01j]


In [15]:
import numpy as np

def compute_doppler_shift(v, fc):
    """
    根据移动速度 v (m/s) 和载波频率 fc (Hz) 计算多普勒频移
    """
    c = 3e8  # 光速 m/s
    return (v / c) * fc

def add_doppler_shift(signal, fd, fs):
    """
    给IQ信号加多普勒频移
    signal: shape (num_channels, num_samples)，复数IQ信号
    fd: 多普勒频移 (Hz)
    fs: 采样率 (Hz)
    """
    num_channels, num_samples = signal.shape
    t = np.arange(num_samples) / fs  # 时间轴
    doppler_phase = np.exp(1j * 2 * np.pi * fd * t)  # 复指数
    return signal * doppler_phase

# 例子
fs = 20e6  # 采样率，比如1 MHz
fc = 2.4e9  # 载波频率，比如2.4 GHz Wi-Fi
v = 120  # 移动速度，比如30 m/s (~108 km/h)

fd = compute_doppler_shift(v, fc)  # 先计算多普勒频移
print(f"计算得到的多普勒频移 fd = {fd:.2f} Hz")

# 假设 data 是你之前读出来的list，每个是 shape (信号数量, 采样点数)
data_with_doppler = []
for sig in noisy_data:
    shifted_sig = add_doppler_shift(sig, fd, fs)
    data_with_doppler.append(shifted_sig)

print("添加多普勒频移后的信号：")
print(data_with_doppler)


计算得到的多普勒频移 fd = 960.00 Hz
添加多普勒频移后的信号：
[array([[ 3.94528279e-02-5.98180779e-03j,  2.08482819e-02+1.05903859e-03j,
        -1.44894704e-01+6.22384436e-03j, ...,
        -2.91944743e-01-2.60583784e-01j, -2.27969053e-01-1.37459736e+00j,
        -7.35402799e-02+7.43593105e-01j],
       [-1.19409651e-01-6.65819948e-03j,  1.55643757e-01+1.11289039e-03j,
         1.38028592e-01+5.83028615e-03j, ...,
        -2.26756515e-01-8.70104355e-01j,  5.49005152e-01-1.12106541e+00j,
        -7.71323592e-01+8.10444485e-01j],
       [ 1.36144977e-01+3.14398823e-03j, -4.58483497e-02-1.65639371e-03j,
         6.81207503e-04-4.13448585e-03j, ...,
        -1.62432474e-01+1.09704494e+00j, -6.16036428e-01+7.64641832e-01j,
         1.15662263e+00-7.73762950e-01j],
       ...,
       [ 9.86020085e-02+1.49593753e-03j,  1.37967396e-01-5.92744763e-04j,
        -4.58824794e-02+2.35559866e-02j, ...,
        -1.06145623e+00+1.07161118e+00j, -4.69037187e-01+1.27828189e-01j,
         1.44428573e+00-7.30444535e-01j],
    

In [16]:
from torch.utils.data import DataLoader, TensorDataset, random_split, Subset
import numpy as np
import torch

# === 拼接数据 ===
all_data = []
all_labels = []

for device_idx, device_signals in enumerate(data_with_doppler):
    device_labels = np.full((device_signals.shape[0],), device_idx)
    all_data.append(torch.tensor(device_signals, dtype=torch.float32))
    all_labels.append(torch.tensor(device_labels, dtype=torch.long))

all_data = torch.cat(all_data, dim=0)
all_labels = torch.cat(all_labels, dim=0)

print(f"最终数据 shape: {all_data.shape}")
print(f"最终标签 shape: {all_labels.shape}")

dataset = TensorDataset(all_data, all_labels)

# === 固定划分训练集 + 测试集 ===
train_size = int(0.8 * len(dataset))  # 80% 训练
test_size = len(dataset) - train_size # 20% 测试
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

print(f"训练集大小: {len(train_dataset)}")
print(f"测试集大小: {len(test_dataset)}")


最终数据 shape: torch.Size([138904, 320])
最终标签 shape: torch.Size([138904])
训练集大小: 111123
测试集大小: 27781


In [17]:
# ==== 数据增强：对训练集添加高斯噪声 ====

def add_noise_to_dataset(dataset, noise_std=0.01):
    noisy_data = []
    noisy_labels = []
    for x, y in dataset:
        noise = torch.randn_like(x) * noise_std
        noisy_x = x + noise
        noisy_data.append(noisy_x)
        noisy_labels.append(y)
    noisy_data = torch.stack(noisy_data)
    noisy_labels = torch.tensor(noisy_labels)
    return TensorDataset(noisy_data, noisy_labels)

train_dataset = add_noise_to_dataset(train_dataset, noise_std=0.01)  # 可调节 std

In [18]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix
import seaborn as sns
from torch.utils.data import DataLoader, Subset
from datetime import datetime
from tqdm import tqdm
from sklearn.model_selection import KFold

# 检查 GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 假设 SNR_dB 和 fd 已经在之前定义
SNR_dB = globals().get('SNR_dB', 'no')
fd = globals().get('fd', 'no')

# 初始化模型参数
input_dim = 320
num_heads = 2
num_layers = 3
num_classes = 20
dropout = 0.5

# 训练参数
batch_size = 256
num_epochs = 200
learning_rate = 1e-4
weight_decay = 1e-4
patience = 10

timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
script_name = "los_DCTF"

folder_name = f"{timestamp}_{script_name}_SNR{SNR_dB}dB_fd{fd}_classes_{num_classes}_Transformer"
save_folder = os.path.join(os.getcwd(), "training_results", folder_name)
os.makedirs(save_folder, exist_ok=True)

results_file = os.path.join(save_folder, "results.txt")
with open(results_file, "w") as f:
    f.write(f"=== Experiment Summary ===\n")
    f.write(f"Feature Folder: trajectory_plots\n")
    f.write(f"Timestamp: {timestamp}\n")
    f.write(f"Total Classes: {num_classes}\n")
    f.write(f"SNR: {SNR_dB} dB\n")
    f.write(f"fd (Doppler shift): {fd} Hz\n")

# 定义SignalTransformer模型
class SignalTransformer(nn.Module):
    def __init__(self, input_dim, num_heads, num_layers, num_classes, dropout=0.1):
        super(SignalTransformer, self).__init__()
        self.embedding = nn.Linear(input_dim, input_dim)
        self.transformer = nn.Transformer(
            d_model=input_dim,
            nhead=num_heads,
            num_encoder_layers=num_layers,
            dropout=dropout,
            batch_first=True
        )
        self.fc = nn.Linear(input_dim, num_classes)
    
    def forward(self, x):
        x = self.embedding(x)
        x = self.transformer(x, x)
        if len(x.shape) == 3:
            x = x[:, -1, :]
        x = self.fc(x)
        return x

# K折交叉验证
n_splits = 5
kfold = KFold(n_splits=n_splits, shuffle=True, random_state=42)
fold_results = []
test_results = []

test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

for fold, (train_idx, val_idx) in enumerate(kfold.split(train_dataset)):
    print(f"\n====== Fold {fold+1}/{n_splits} ======")
    train_subset = Subset(train_dataset, train_idx)
    val_subset = Subset(train_dataset, val_idx)

    train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True, drop_last=True)
    val_loader = DataLoader(val_subset, batch_size=batch_size, shuffle=False, drop_last=True)

    model = SignalTransformer(input_dim=input_dim, num_heads=num_heads,
                              num_layers=num_layers, num_classes=num_classes,
                              dropout=dropout).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

    train_losses, val_losses = [], []
    train_accuracies, val_accuracies = [], []

    best_val_acc = 0
    patience_counter = 0
    best_model_wts = None
    best_val_loss = float('inf')
    patience_counter = 0

    for epoch in range(num_epochs):
        model.train()
        running_train_loss = 0.0
        correct_train, total_train = 0, 0
        
        with tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", unit="batch") as tepoch:
            for batch_idx, (inputs, labels) in enumerate(tepoch):
                inputs, labels = inputs.to(device), labels.to(device)
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                
                running_train_loss += loss.item()
                _, predicted = torch.max(outputs, 1)
                total_train += labels.size(0)
                correct_train += (predicted == labels).sum().item()
                train_accuracy = 100 * correct_train / total_train
                tepoch.set_postfix(train_loss=running_train_loss / (batch_idx + 1), train_accuracy=train_accuracy)

        epoch_train_loss = running_train_loss / len(train_loader)
        epoch_train_acc = 100 * correct_train / total_train
        train_losses.append(epoch_train_loss)
        train_accuracies.append(epoch_train_acc)

        # 验证
        model.eval()
        running_val_loss = 0.0
        correct_val, total_val = 0, 0
        all_preds, all_labels = [], []

        with torch.no_grad():
            with tqdm(val_loader, desc="Validation", unit="batch") as vepoch:
                for val_inputs, val_labels in vepoch:
                    val_inputs, val_labels = val_inputs.to(device), val_labels.to(device)
                    val_outputs = model(val_inputs)
                    val_loss = criterion(val_outputs, val_labels)
                    running_val_loss += val_loss.item()
                    _, val_predicted = torch.max(val_outputs, 1)
                    total_val += val_labels.size(0)
                    correct_val += (val_predicted == val_labels).sum().item()
                    val_accuracy = 100 * correct_val / total_val
                    vepoch.set_postfix(val_loss=running_val_loss / (batch_idx + 1), val_accuracy=val_accuracy)


                    # 保存混淆矩阵数据
                    all_preds.extend(val_predicted.cpu().numpy())
                    all_labels.extend(val_labels.cpu().numpy())

        epoch_val_loss = running_val_loss / len(val_loader)
        epoch_val_acc = 100 * correct_val / total_val
        val_losses.append(epoch_val_loss)
        val_accuracies.append(epoch_val_acc)

        print(f"Epoch {epoch+1}/{num_epochs}: Train Loss: {epoch_train_loss:.4f}, Train Acc: {epoch_train_acc:.2f}%, Val Loss: {epoch_val_loss:.4f}, Val Acc: {epoch_val_acc:.2f}%")
        with open(results_file, "a") as f:
            f.write(f"Epoch {epoch+1} | Train Loss: {epoch_train_loss:.4f} | Train Acc: {epoch_train_acc:.2f}% | Val Loss: {epoch_val_loss:.4f} | Val Acc: {epoch_val_acc:.2f}%\n")

        # Early stopping
        if epoch_val_loss < best_val_loss:
            best_val_loss = epoch_val_loss
            best_val_acc = epoch_val_acc   # 更新最佳验证精度
            best_model_wts = model.state_dict()  # 保存最佳参数
            patience_counter = 0
        else:
            patience_counter += 1

        if patience_counter >= patience:
            print(f"Early stopping at epoch {epoch+1}")
            break

        scheduler.step()

    # ========= 恢复最优模型并测试 =========
    model.load_state_dict(best_model_wts)
    model.eval()

    # --- 在最优模型上重新跑验证集，得到混淆矩阵 ---
    all_preds, all_labels = [], []
    correct_val, total_val = 0, 0
    with torch.no_grad():
        for val_inputs, val_labels in val_loader:
            val_inputs, val_labels = val_inputs.to(device), val_labels.to(device)
            val_outputs = model(val_inputs)
            _, val_predicted = torch.max(val_outputs, 1)
            total_val += val_labels.size(0)
            correct_val += (val_predicted == val_labels).sum().item()
            all_preds.extend(val_predicted.cpu().numpy())
            all_labels.extend(val_labels.cpu().numpy())

    best_val_acc = 100 * correct_val / total_val

    # --- 在最优模型上重新跑测试集 ---
    correct_test, total_test = 0, 0
    test_preds, test_labels_all = [], []
    with torch.no_grad():
        for test_inputs, test_labels in test_loader:
            test_inputs, test_labels = test_inputs.to(device), test_labels.to(device)
            outputs = model(test_inputs)
            _, predicted = torch.max(outputs, 1)
            total_test += test_labels.size(0)
            correct_test += (predicted == test_labels).sum().item()
            test_preds.extend(predicted.cpu().numpy())
            test_labels_all.extend(test_labels.cpu().numpy())

    test_acc = 100 * correct_test / total_test

    print(f"Fold {fold+1}: Best Val Acc={best_val_acc:.2f}%, Test Acc={test_acc:.2f}%")

    fold_results.append(best_val_acc)
    test_results.append(test_acc)

    # --- 绘制混淆矩阵（基于最优模型的验证集）---
    cm = confusion_matrix(all_labels, all_preds)
    plt.figure(figsize=(10,8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.xlabel('Predicted'); plt.ylabel('True'); plt.title(f'Fold {fold+1} Validation Confusion Matrix')
    plt.tight_layout()
    plt.savefig(os.path.join(save_folder, f"fold_{fold+1}_val_confusion_matrix.png"))
    plt.close()
    np.savetxt(os.path.join(save_folder, f"fold_{fold+1}_val_confusion_matrix.txt"), cm, fmt='%d')

    # --- 测试集混淆矩阵 ---
    cm_test = confusion_matrix(test_labels_all, test_preds)
    plt.figure(figsize=(10,8))
    sns.heatmap(cm_test, annot=True, fmt='d', cmap='Blues')
    plt.xlabel('Predicted'); plt.ylabel('True'); plt.title(f'Fold {fold+1} Test Confusion Matrix')
    plt.tight_layout()
    plt.savefig(os.path.join(save_folder, f"fold_{fold+1}_test_confusion_matrix.png"))
    plt.close()
    np.savetxt(os.path.join(save_folder, f"fold_{fold+1}_test_confusion_matrix.txt"), cm_test, fmt='%d')

    # 绘图
    plt.figure()
    plt.plot(range(1, len(train_losses)+1), train_losses, label='Train Loss')
    plt.plot(range(1, len(val_losses)+1), val_losses, label='Val Loss')
    plt.xlabel('Epoch'); plt.ylabel('Loss'); plt.legend()
    plt.title(f'Fold {fold+1} Loss Curve')
    plt.savefig(os.path.join(save_folder, f"fold_{fold+1}_loss_curve.png"))
    plt.close()

    plt.figure()
    plt.plot(range(1, len(train_accuracies)+1), train_accuracies, label='Train Acc')
    plt.plot(range(1, len(val_accuracies)+1), val_accuracies, label='Val Acc')
    plt.xlabel('Epoch'); plt.ylabel('Accuracy'); plt.legend()
    plt.title(f'Fold {fold+1} Accuracy Curve')
    plt.savefig(os.path.join(save_folder, f"fold_{fold+1}_accuracy_curve.png"))
    plt.close()

# 汇总结果
avg_acc = np.mean(fold_results)
avg_test_acc = np.mean(test_results)
with open(results_file, "a") as f:
    for i in range(n_splits):
        f.write(f"Fold {i+1}: Val Acc={fold_results[i]:.2f}%, Test Acc={test_results[i]:.2f}%\n")
    f.write(f"\nAverage Val Accuracy: {avg_acc:.2f}%\n")
    f.write(f"Average Test Accuracy: {avg_test_acc:.2f}%\n")

print(f"\nAverage Val Acc={avg_acc:.2f}%, Average Test Acc={avg_test_acc:.2f}%")


Using device: cuda

====== Fold 1/5 ======


Validation: 100%|██████████| 86/86 [00:00<00:00, 136.98batch/s, val_accuracy=9.64, val_loss=0.734] 


Epoch 1/200: Train Loss: 3.0185, Train Acc: 7.45%, Val Loss: 2.9623, Val Acc: 9.64%


Validation: 100%|██████████| 86/86 [00:00<00:00, 124.81batch/s, val_accuracy=9.64, val_loss=0.733]


Epoch 2/200: Train Loss: 2.9964, Train Acc: 7.89%, Val Loss: 2.9566, Val Acc: 9.64%


...

Epoch 40/200: Train Loss: 0.0159, Train Acc: 99.50%, Val Loss: 0.0314, Val Acc: 99.21%


Validation: 100%|██████████| 86/86 [00:00<00:00, 160.44batch/s, val_accuracy=99.2, val_loss=0.00704]


Epoch 41/200: Train Loss: 0.0138, Train Acc: 99.59%, Val Loss: 0.0284, Val Acc: 99.22%


Validation: 100%|██████████| 86/86 [00:00<00:00, 137.36batch/s, val_accuracy=99.3, val_loss=0.00697]


Epoch 42/200: Train Loss: 0.0130, Train Acc: 99.60%, Val Loss: 0.0281, Val Acc: 99.26%
Early stopping at epoch 42
Fold 5: Best Val Acc=99.26%, Test Acc=99.22%

Average Val Acc=99.20%, Average Test Acc=99.16%
